# Machine Learning at CoDaS-HEP 2024, Lesson 3: Main Project

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.datasets
import torch
from torch import nn
from torch import optim

<br><br><br><br><br>

## Project 1: classify jets with a neural network

### Step 0: Put your name on the results sheet

Go to the [Google Spreadsheet for this course](https://docs.google.com/spreadsheets/d/1nRtNJoxW1i-jCr04ZHUlfv0DU4tMGbyvCZcpXakYedE/edit?usp=sharing) and add your name to the _second_ sheet (tab on the bottom of the window):

<a href="https://docs.google.com/spreadsheets/d/1nRtNJoxW1i-jCr04ZHUlfv0DU4tMGbyvCZcpXakYedE/edit?usp=sharing"><img src="../img/google-sheet-main-project.png" width="700"></a>

Your results (a ROC curve) will go _below_ your name. You own a column of this spreadsheet.

<br><br><br><br><br>

### Step 1: Download and understand the data

We'll use an LHC dataset from an online catalog, [hls4ml_lhc_jets_hlf](https://openml.org/search?type=data&status=active&id=42468).

The full description is online, with references to the paper in which it was published.

Scikit-Learn has a tool for downloading it (takes a minute or two).

In [ ]:
hls4ml_lhc_jets_hlf = sklearn.datasets.fetch_openml("hls4ml_lhc_jets_hlf")

features, targets = hls4ml_lhc_jets_hlf["data"], hls4ml_lhc_jets_hlf["target"]

View the features (16 numerical properties of jets) as a Pandas DataFrame:

In [ ]:
features

And some summary statistics for each feature:

In [ ]:
features.describe()

You can convert the (830000 row × 16 column) DataFrame into a NumPy array (of shape `(830000, 16)`) with

In [ ]:
features.values

<br><br><br><br><br>

View the target (5 jet categories) as a Pandas Series:

In [ ]:
targets

The categories are represented as 5 Python strings (`dtype='object'` means Python objects in a NumPy array/Pandas Series).

In [ ]:
targets.cat.categories

But the large dataset consists of (8-bit) integers corresponding to the position in this list of categories.

In [ ]:
targets.cat.codes

In [ ]:
targets.cat.codes.values

<br><br><br><br><br>

As for the physical meaning of these features and targets, there's more in the paper, but

* `'g'` means gluon jet (a gluon from the original proton-proton collision hadronized into a jet)
* `'q'` means a light quark hadronized into a jet: up (u), down (d), or strange (s)
* `'t'` means a top (t) quark decayed into a bottom (b) quark and a W boson, which subsequently decayed and hadronized
* `'W'` means a W boson directly from the original proton-proton collision decayed and its constituents hadronized
* `'Z'` means the same thing for a Z boson

<img src="../img/JetDataset.png" width="429">

Each of these physical processes produces systematically different jet shapes, as characterized by the 16 input features.

The distributions of these jet shapes overlap, so this is a job for machine learning! (Not just manually chosen cuts, for instance.)

Use the space below for any plotting. It's always good to look at the data (in _some_ way) before trying to fit it.

<br><br><br><br><br>

### Step 2: Split the data into training, validation, and test samples

For this exercise, put

* 80% of the data into the training sample, which the optimizer will use in its fits
* 10% of the data into the validation sample, which you will look at while developing the model
* 10% of the data into the test sample, which you should not look at until you're done and making the final ROC curve

These data are supposed to be Independent and Identically Distributed (IID), but just in case there are any beginning-of-dataset, end-of-dataset biases, sample them randomly (or at least in an interleaved way).

Scikit-Learn has a [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), PyTorch has a [torch.utils.data.random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split), or you can do it manually.

<br><br><br><br><br>

### Step 3: Build a classifier neural network

It should have the following architecture:

* take the 16 numerical features as input
* have 3 (fully connected) hidden layers with 32 neurons each
* return probabilities for the 5 output categories

Use any tools you have to improve the quality of the model, but the model should be implemented in PyTorch.

Think about all of the issues covered in [Lesson 2: Issues in Practice](../lesson-2-issues/lecture-slides.ipynb).

If you use [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (that's not the only way!), remember that it applies a softmax to predicted values you give it, so your model can't also apply a softmax, and it can take the target values as a 1-dimensional array of integers (the true category) or as a 2-dimensional array of one-hot vectors (the category probabilities, which are all 0's and 1's).

Since all of the examples in Lesson 2 had small datasets, we used an excessive number of epochs. For this large dataset, you shouldn't need more than 10 epochs, and it can be useful to debug your model one epoch at a time. Print out useful information in the loop over epochs, so that you don't have to wait for the whole thing to finish.

Helpful hint: are the input data close to the $(-1, 1)$ interval? If not, what can you do?

<br><br><br><br><br>

### Step 4: Monitor the loss function

Plot the loss function versus epoch for the training sample and the validation sample (and _not_ the test sample!).

Do they diverge? If so, what can you do about that?

Suggestion: compute the validation loss before the training loss, since a training loop over mini-batches changes the model state.

Helpful hint: since you'll be comparing losses computed from different dataset sizes, you need to scale them by the number of data points.

```python
for epoch in range(NUM_EPOCHS):
    features_tensor, targets_tensor = the_validation_sample
    ...
    validation_loss = loss.item() * len(the_validation_sample) * (0.8 / 0.1)

    training_loss = 0
    for features_tensor, targets_tensor in training_sample_batches:
        ...
        training_loss += loss.item() * len(targets_tensor)   # one training mini-batch
```

<br><br><br><br><br>

### Step 5: Compute a 5×5 confusion matrix

We now have 5 categories, rather than 2, so the confusion matrix is 5×5:

| | predicted `'g'` | predicted `'q'` | predicted `'t'` | predicted `'w'` | predicted `'z'` |
|--:|:--:|:--:|:--:|:--:|:--:|
| **true `'g'`** | # | # | # | # | # |
| **true `'q'`** | # | # | # | # | # |
| **true `'t'`** | # | # | # | # | # |
| **true `'w'`** | # | # | # | # | # |
| **true `'z'`** | # | # | # | # | # |

Each prediction from your model is a vector of 5 numbers. The softmax of these 5 numbers are the 5 probabilities for each category (guaranteed by the [definition of softmax](https://en.wikipedia.org/wiki/Softmax_function) to add up to 1). If you use [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), your model does not apply the softmax, so you'd need to apply it as an additional step.

For a large set of predictions (a 2-dimensional array with shape `(num_predictions, 5)`), the [torch.argmax](https://pytorch.org/docs/stable/generated/torch.argmax.html#torch.argmax) with `axis=1` finds the index of the maximum probability in each prediction.

Use this to count how many true `'g'` your model predicts as `'g'`, how many true `'g'` your model predicts as `'q'`, etc. for all 25 elements of the confusion matrix.

Then plot this matrix as a colormap using Matplotlib's [ax.imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html).

Use the validation sample only!

<br><br><br><br><br>

### Step 6: Compute a 2×2 confusion matrix

Suppose that we're primarily interested in how the model separates lightweight QCD jets (`'g'` and `'q'`) from heavy electroweak jets (`'t'`, `'w'`, `'z'`). Since the categories are

In [ ]:
targets.cat.categories

the index range `0:2` selects lightweight QCD probabilities and the index range `2:5` selects heavy electroweak probabilities. By summing over one of these ranges (slice the second dimension of your set of predicted probabilities, followed by [torch.sum](https://pytorch.org/docs/stable/generated/torch.sum.html#torch.sum) with `axis=1`), you get its total probability $p$, and the other range gives $1 - p$.

Compute the 2×2 confusion matrix for the problem of distinguishing lightweight QCD jets (background) from heavy electroweak jets (signal). You might find this easiest to write as a function of a `threshold`, with the default `threshold` being `0.5`. Anyway, you'll need that for the next step.

Finally, plot a ROC curve, the true positive rate versus false positive rate curve. How close does your model get to perfection (true positive rate = 1 and false positive rate = 0)? How does its shape differ at one end of the ROC curve from the other? Would it be symmetric if you swapped true positive rate withs false positive rate?

<br><br><br><br><br>

### Step 7: Make a final ROC curve with the test sample and submit it

When you're satisfied with the ROC curve on the validation sample, switch to the test sample and plot it again.

Then use this function:

In [ ]:
def resample_loss_curve(false_positive_rates, true_positive_rates):
    series = pd.Series(true_positive_rates, index=false_positive_rates).groupby(level=0).mean()
    new_x = np.linspace(0, 1, 100)**2
    new_y = series.reindex(series.index.union(new_x)).interpolate("values").loc[new_x]

    fig, ax = plt.subplots(figsize=(6, 6))

    ax.plot(new_x, new_y, marker=".")
    ax.plot(false_positive_rates, true_positive_rates)

    ax.grid(True, linestyle=":")

    ax.set_xlabel("false positive rate")
    ax.set_ylabel("true positive rate")

    print("Copy-paste the the following numbers into your column in the spreadsheet:")
    print("https://docs.google.com/spreadsheets/d/1nRtNJoxW1i-jCr04ZHUlfv0DU4tMGbyvCZcpXakYedE/edit?usp=sharing")
    print()
    print("\n".join(str(x) for x in new_y))
    print()
    print("Also, be sure that these two curves exactly overlap each other:")

on your `false_positive_rates` list and your `true_positive_rates` list to resample it and print out 100 values to copy-paste into the [Google Spreadsheet for this course](https://docs.google.com/spreadsheets/d/1nRtNJoxW1i-jCr04ZHUlfv0DU4tMGbyvCZcpXakYedE/edit?usp=sharing) (_second_ sheet, "Main Project").

You have already claimed a column in this spreadsheet with your name. Paste the 100 values in the column under your name.

Scroll down below everyone's data to see the overlay of ROC curves. How did your model do? (There is no reason to think that mine will be the best; yours might be closer to the "perfect model" corner!)

<img src="../img/only-one-ROC-curve.png" width="700">

<br><br><br><br><br>

## Project 2: Help others

If you finished before the end of the session, get up and walk around the room to see if anyone else needs help.

Finishing early means that you understand the concepts and tools well enough to perform an analysis. But do you understand the concepts and tools well enough to explain them to someone who doesn't? That's the next-level stage of understanding.